In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

## Read Data

In [21]:
csv_file='./data/players_%s.csv'
years = ['16', '17', '18', '19', '20']

fifa_total = pd.DataFrame()
for year in years:
    fifa_data = pd.read_csv(csv_file % year)
    fifa_data['year'] = '20%s' % year
    fifa_total = fifa_total.append(fifa_data)
    
fifa_total.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,year
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,28,1987-06-24,170,72,Argentina,FC Barcelona,...,57+3,57+3,57+3,62+3,57+3,44+3,44+3,44+3,57+3,2016
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,30,1985-02-05,185,80,Portugal,Real Madrid,...,60+3,60+3,60+3,64+3,60+3,52+3,52+3,52+3,60+3,2016
2,9014,https://sofifa.com/player/9014/arjen-robben/16...,A. Robben,Arjen Robben,31,1984-01-23,180,80,Netherlands,FC Bayern München,...,60+3,60+3,60+3,65+3,59+3,47+3,47+3,47+3,59+3,2016
3,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Neuer,29,1986-03-27,193,92,Germany,FC Bayern München,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
4,176580,https://sofifa.com/player/176580/luis-suarez/1...,L. Suárez,Luis Alberto Suárez Díaz,28,1987-01-24,182,85,Uruguay,FC Barcelona,...,65+3,65+3,65+3,67+3,64+3,58+3,58+3,58+3,64+3,2016


### Define useful information (eg. players position group, useful columns)

In [22]:
forward = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw']
mid = ['lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'ldm', 'cdm', 'rdm']
back = ['lwb', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb']
keeper = ['gk']

In [23]:
player_info_col = ['sofifa_id', 'player_url', 'short_name', 'age', 'height_cm', 'weight_kg', 
                   'nationality', 'club', 'overall', 'potential', 'value_eur', 'wage_eur', 
                   'player_positions', 'preferred_foot']

In [28]:
player_abilities_col = ['pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 
                        'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed', 'gk_positioning', 
                        'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 
                        'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy',
                        'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 
                        'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 
                        'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 
                        'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 
                        'mentality_composure', 'defending_marking', 'defending_standing_tackle', 'defending_sliding_tackle',
                        'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning', 
                        'goalkeeping_reflexes']

### Handle Missing Data

In [29]:
fifa_total[player_abilities_col] = fifa_total[player_abilities_col].replace(np.NaN, 0)
fifa_total[player_abilities_col].head()

,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,92.0,88.0,86.0,95.0,24.0,62.0,0.0,0.0,0.0,0.0,...,74,0,13-12,23+2,21+1,6,11,15,14,8
1,92.0,93.0,80.0,91.0,33.0,78.0,0.0,0.0,0.0,0.0,...,85,0,22,31,23,7,11,15,14,11
2,92.0,86.0,82.0,92.0,32.0,64.0,0.0,0.0,0.0,0.0,...,80,0,29,26,26,10,8,11,5,15
3,0.0,0.0,0.0,0.0,0.0,0.0,85.0,87.0,91.0,86.0,...,37,0,10-15,10-15,11-14,85-3,87+2,91,90,86
4,83.0,88.0,79.0,87.0,42.0,79.0,0.0,0.0,0.0,0.0,...,85,0,30,45,38,27,25,31,33,37


### Handle +/- data

In [38]:
def calculate_diff(x):
    if isinstance(x, str):
        if "+" in x:
            x_split = x.split("+")
            x_new = int(x_split[0]) + int(x_split[1])
            return x_new
        elif "-" in x:
            x_split = x.split("-")
            x_new = int(x_split[0]) + int(x_split[1])
            return x_new
        else:
            return x
    else:
        return x

for col in player_abilities_col:
    fifa_total[col] = fifa_total[col].apply(calculate_diff)
fifa_total[player_abilities_col].head()

,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,92.0,88.0,86.0,95.0,24.0,62.0,0.0,0.0,0.0,0.0,...,74,0,25,25,22,6,11,15,14,8
1,92.0,93.0,80.0,91.0,33.0,78.0,0.0,0.0,0.0,0.0,...,85,0,22,31,23,7,11,15,14,11
2,92.0,86.0,82.0,92.0,32.0,64.0,0.0,0.0,0.0,0.0,...,80,0,29,26,26,10,8,11,5,15
3,0.0,0.0,0.0,0.0,0.0,0.0,85.0,87.0,91.0,86.0,...,37,0,25,25,25,88,89,91,90,86
4,83.0,88.0,79.0,87.0,42.0,79.0,0.0,0.0,0.0,0.0,...,85,0,30,45,38,27,25,31,33,37


#### Age Vs Performance

In [41]:
from sklearn.preprocessing import MinMaxScaler

performance_col = ['overall', 'potential', 'value_eur', 'wage_eur']
performance_col.extend(player_abilities_col)

data_y = fifa_total.loc[:, performance_col]
data_x = fifa_total.loc[:, 'age']

In [44]:
scaler_y = MinMaxScaler()
data_y_standard = scaler_y.fit_transform(data_y)
data_y_standard = pd.DataFrame(data_y_standard, columns=performance_col)
data_y_standard.head()

,overall,potential,value_eur,wage_eur,pace,shooting,passing,dribbling,defending,physic,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,1.00,1.000000,0.902439,0.973451,0.958333,0.946237,0.924731,0.989583,0.263736,0.673913,...,0.621622,0.0,0.210526,0.205607,0.137681,0.051546,0.097087,0.127273,0.136842,0.073684
1,0.98,0.960784,0.695122,0.840708,0.958333,1.000000,0.860215,0.947917,0.362637,0.847826,...,0.720721,0.0,0.184211,0.261682,0.144928,0.061856,0.097087,0.127273,0.136842,0.105263
2,0.92,0.901961,0.455285,0.442478,0.958333,0.924731,0.881720,0.958333,0.351648,0.695652,...,0.675676,0.0,0.245614,0.214953,0.166667,0.092784,0.067961,0.090909,0.042105,0.147368
3,0.92,0.901961,0.471545,0.442478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.288288,0.0,0.210526,0.205607,0.159420,0.896907,0.854369,0.818182,0.936842,0.894737
4,0.92,0.901961,0.560976,0.530973,0.864583,0.946237,0.849462,0.906250,0.461538,0.858696,...,0.720721,0.0,0.254386,0.392523,0.253623,0.268041,0.233010,0.272727,0.336842,0.378947
